<h2 style="font-weight: bold">House Prices Competition</h2>

<h4>This is my second published notebook on Kaggle, So yeah no wonder it's about the House Prices Competition 😄😄<br><br>I will be doing a simple then advanced EDA, Data Visualization and Pre-Processing. I also will test different approaches and regression techniques to improve my score.<br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>


In [1]:
# Required packages
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
train.head()

#  **Exploratory Data Analysis**

In [4]:
# Getting to know data
print(f"Training Set :\n Number of rows : {train.shape[0]}, Number of Columns : {train.shape[1]}")
print(f"Test Set :\n Number of rows : {test.shape[0]}, Number of Columns : {test.shape[1]}")

In [5]:
# extract numeric data
num_cols = train.loc[:,train.dtypes != 'object'].drop(['Id'], axis=1).columns
num_train = train[num_cols]
# extract categorical data
cat_cols = train.loc[:,train.dtypes == 'object'].columns
cat_train = train[cat_cols]

print("Total Numerical Cols : ", len(num_cols))
print("Total Categorical Cols : ", len(cat_cols))

In [6]:
# summary of numerical variable
train.describe()

In [7]:
# summary of categorial variable
train.info()

In [8]:
# let's clean visualizations :)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
# Let's take a look at the distribution of the SalePrice 
sns.distplot(train['SalePrice'] , fit=norm);
(mu, sigma) = norm.fit(train['SalePrice'])
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

In [10]:
price = np.log1p(train["SalePrice"])
 
sns.distplot(price , fit=norm);
(mu, sigma) = norm.fit(price)
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')